In [105]:
!pip install pyngrok

In [106]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/student_project_Lead_time_prediction_data.csv")
#data.isnull().any()
#sepearting data
data1=data.iloc[:,1:11]
data1.head()
#data1.isnull().sum()
data1=data1.dropna(subset='CATALOG_NO')
data1=data1.dropna(subset='INDENT_DT')
data1=data1.dropna(subset='MATL_RECEIPT_DT')
data1=data1.dropna(subset='INSUR_ITEM_IND')
data1=data1.dropna(subset='AR_ITEM_IND')
data1=data1.dropna(subset='SPC_ITEM_IND')
data1=data1.dropna(subset='QUALITY_IND')
data1=data1.dropna(subset='FTP_IND')
data1=data1.dropna(subset='FSN_IND')
data1.isnull().sum()
data1.head()
data2=data1
data2.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-106-4a1638a2f642>:5: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/content/drive/MyDrive/student_project_Lead_time_prediction_data.csv")


,CATALOG_NO,CATALOG_DESC,INDENT_DT,MATL_RECEIPT_DT,INSUR_ITEM_IND,AR_ITEM_IND,SPC_ITEM_IND,QUALITY_IND,FTP_IND,FSN_IND
0,209110603,BRG.NO.32211 A,1/21/2002,9/14/2002,N,N,Y,N,N,F
1,2585401002,WHITE CANVAS SHOES SIZE: 10.,6/10/2002,9/24/2002,N,N,N,N,N,S
2,910995101,NYLON WHITE SOCKS. .,6/10/2002,9/24/2002,N,N,N,N,N,S
3,1132805505,UNILITE MAKE 1X250W HPSV CONTROL GEAR BOX DULY...,3/9/2002,9/6/2002,N,N,Y,N,N,F
4,209130407,BRG.NO. 32213 J2/Q,5/3/2002,9/14/2002,N,N,Y,N,N,F


In [107]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data2['INSUR_ITEM_IND']=le.fit_transform(data2['INSUR_ITEM_IND'])
data2['AR_ITEM_IND']=le.fit_transform(data2['AR_ITEM_IND'])
data2['SPC_ITEM_IND']=le.fit_transform(data2['SPC_ITEM_IND'])
data2['QUALITY_IND']=le.fit_transform(data2['QUALITY_IND'])
data2['FTP_IND']=le.fit_transform(data2['FTP_IND'])
data2['FSN_IND']=le.fit_transform(data2['FSN_IND'])
data2.head()

,CATALOG_NO,CATALOG_DESC,INDENT_DT,MATL_RECEIPT_DT,INSUR_ITEM_IND,AR_ITEM_IND,SPC_ITEM_IND,QUALITY_IND,FTP_IND,FSN_IND
0,209110603,BRG.NO.32211 A,1/21/2002,9/14/2002,1,0,1,0,0,0
1,2585401002,WHITE CANVAS SHOES SIZE: 10.,6/10/2002,9/24/2002,1,0,0,0,0,2
2,910995101,NYLON WHITE SOCKS. .,6/10/2002,9/24/2002,1,0,0,0,0,2
3,1132805505,UNILITE MAKE 1X250W HPSV CONTROL GEAR BOX DULY...,3/9/2002,9/6/2002,1,0,1,0,0,0
4,209130407,BRG.NO. 32213 J2/Q,5/3/2002,9/14/2002,1,0,1,0,0,0


In [108]:
data2['INDENT_DT'] = pd.to_datetime(data['INDENT_DT'],errors='coerce')
data2['MATL_RECEIPT_DT'] = pd.to_datetime(data['MATL_RECEIPT_DT'],errors='coerce')
data2['time_difference'] = (data2['MATL_RECEIPT_DT'] - data2['INDENT_DT']).dt.total_seconds()
data2['time_difference'] = data2['time_difference'] / (24 * 60 * 60)
data2.head()
data2['time_difference'].fillna(data2['time_difference'].mean(),inplace=True)
data2=data2.dropna(subset='INDENT_DT')
data2=data2.dropna(subset='MATL_RECEIPT_DT')
data2.isnull().sum()
data3=data2
data3.drop('CATALOG_DESC',axis=1, inplace=True)

In [115]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import SVC
from sklearn import metrics

X = data3[['CATALOG_NO']]
y = data3['time_difference']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
clf=DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)
clf.fit(X_train, y_train)


DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)

In [122]:
from flask import Flask
from pyngrok import ngrok

In [123]:
ngrok.set_auth_token("2W0mIhzmyEa7zN3NKSlDmwbsoqZ_5zZFZigFju6jYEDy7S5ts")
public_url=ngrok.connect(5000).public_url

In [114]:
port_no=5000

In [ ]:
from flask import Flask, request, render_template, jsonify
app = Flask(__name__)
ngrok.set_auth_token("your_token")
public_url =  ngrok.connect(port_no).public_url
@app.route("/")
def home():
    return render_template('index.html')

@app.route('/getprediction',methods=['POST'])

def getprediction():
    try:
        data = request.get_json()
        catalog_no = data.get('CATALOG_NO')
        order_date = data.get('ORDER_DATE')

        # Perform prediction using your 'clf' model
        # You need to preprocess the input data and use your model for prediction here
        # Replace the following line with your actual prediction logic
        prediction = clf.predict([[catalog_no]])

        return jsonify({'y_pred': int(prediction[0])}) # Assuming prediction is a single value
    except Exception as e:
        return jsonify({'error': str(e)})



print(f"To acces the Gloable link please click {public_url}")



app.run(port=port_no)

To acces the Gloable link please click https://f1c8-34-168-192-147.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Sep/2023 21:37:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2023 21:37:37] "GET / HTTP/1.1" 200 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [28/Sep/2023 21:37:52] "POST /getprediction HTTP/1.1" 200 -
